In [1]:
import json
from lxml.html import HtmlElement
import lxml.html
from typing import List
import requests


base_url = 'https://help.plemiona.pl/wiki/'


def get_lvl_costs(tree: HtmlElement, col_nr: int) -> List[int]:
    els = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[%d]' % col_nr)
    if len(els) == 1:
        els = tree.xpath('//*[@id="mw-content-text"]/div/table[2]/tbody/tr/td[%d]' % col_nr)
    resource = []
    for el in els:
        try:
            resource += [int(el.text_content().replace('.', ''))]
        except:
            pass

    return resource


def get_costs_from_tree(tree: HtmlElement):
    woods = get_lvl_costs(tree, 2)
    stones = get_lvl_costs(tree, 3)
    irons = get_lvl_costs(tree, 4)

    return (
        woods, stones, irons
    )


def get_building_cost(building: str, end_lvl: int = None, start_lvl: int = None, sum_up: bool = True):
    response = requests.get(base_url + building)
    if response.status_code < 400:
        tree = lxml.html.fromstring(response.text)
        cost = get_costs_from_tree(tree)
        if sum_up:
            cost = (
            sum(cost[0][start_lvl-1:end_lvl+1]),
            sum(cost[1][start_lvl-1:end_lvl+1]),
            sum(cost[2][start_lvl-1:end_lvl+1])
        )
    else:
        print("Bad status: %d" % response.status_code)
        cost = None
    return cost

Getting all costs and saving it to file

In [2]:
# building_names = [
#     'Ratusz', 'Koszary', 'Stajnia', 'Warsztat', 'Kuźnia', 'Plac', 'Rynek',
#     'Tartak', 'Cegielnia', 'Huta żelaza', 'Zagroda', 'Spichlerz', 'Schowek',
#     'Mur obronny', 'Pałac', 'Piedestał', 'Kościół', 'Pierwszy Kościół', 'Wieża Strażnicza']

# building_costs = {}
# for name in building_names:
#     try:
#         building_costs[name] = get_building_cost(name, sum_up=False)
#     except Exception as e:
#         print("Error for %s." % name)
#         print(e)

In [3]:
# with open("data/building_costs.json", "w") as file:
#     json.dump(building_costs, file)

Loading and counting requirenments

In [4]:
with open("data/building_costs.json", "r") as file:
    building_costs = json.load(file)

In [15]:
l_lvl, h_lvl = 17, 20
building = "Huta żelaza"

costs = [
    sum(building_costs[building][0][l_lvl-1:h_lvl]),
    sum(building_costs[building][1][l_lvl-1:h_lvl]),
    sum(building_costs[building][2][l_lvl-1:h_lvl])]
costs

[15806, 23272, 12432]

Cost of building palace

In [6]:
l_lvl, h_lvl = 11, 20
building = "Ratusz"
ratusz = [
    sum(building_costs[building][0][l_lvl-1:h_lvl]),
    sum(building_costs[building][1][l_lvl-1:h_lvl]),
    sum(building_costs[building][2][l_lvl-1:h_lvl])]

l_lvl, h_lvl = 3, 20
building = "Kuźnia"
kuznia = [
    sum(building_costs[building][0][l_lvl-1:h_lvl]),
    sum(building_costs[building][1][l_lvl-1:h_lvl]),
    sum(building_costs[building][2][l_lvl-1:h_lvl])]

l_lvl, h_lvl = 1, 1
building = "Pałac"
palac = [
    sum(building_costs[building][0][l_lvl-1:h_lvl]),
    sum(building_costs[building][1][l_lvl-1:h_lvl]),
    sum(building_costs[building][2][l_lvl-1:h_lvl])]

moneta = [28000, 30000, 25000]

koszt = [r + k + p + m for (r, k, p, m) in zip(ratusz, kuznia, palac, moneta)]
koszt

[159453, 172488, 152101]

Cost of wood for mines

In [7]:
l_lvl, h_lvl = 11, 30
r_id = 1

sum(
    [sum(building_costs['Tartak'][r_id][l_lvl-1:h_lvl]),
    sum(building_costs['Cegielnia'][r_id][l_lvl-1:h_lvl]),
    sum(building_costs['Huta żelaza'][r_id][l_lvl-1:h_lvl])]
)

871902